In [1]:
%pip install -q -i https://pypi.tuna.tsinghua.edu.cn/simple --upgrade tensorflow-cpu chdb pandas pyarrow scikit-learn numpy matplotlib
%pip show tensorflow-cpu chdb

Note: you may need to restart the kernel to use updated packages.
Name: tensorflow-cpu
Version: 2.15.0.post1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/Clickhouse/.venv/lib/python3.9/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 
---
Name: chdb
Version: 1.0.1
Summary: chDB is an in-process SQL OLAP Engine powered by ClickHouse
Home-page: https://github.com/auxten/chdb
Author: auxten
Author-email: auxtenwpc@gmail.com
License: Apache-2.0
Location: /home/Clickhouse/.venv/lib/python3.9/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated pack

In [2]:
import pandas as pd
import zipfile
import urllib.request
import os
import chdb
from chdb import session

# Download and extract the dataset
if not os.path.exists("ml-25m/ratings.csv"):
    url = "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    filehandle, _ = urllib.request.urlretrieve(url)
    zip_file_object = zipfile.ZipFile(filehandle, "r")
    zip_file_object.extractall()

!ls -l ml-25m

total 1129584
-rw-r--r-- 1 root root     10460 Dec 11 08:24 README.txt
-rw-r--r-- 1 root root 435164157 Dec 11 08:24 genome-scores.csv
-rw-r--r-- 1 root root     18103 Dec 11 08:24 genome-tags.csv
-rw-r--r-- 1 root root   1368578 Dec 11 08:24 links.csv
-rw-r--r-- 1 root root   3038099 Dec 11 08:24 movies.csv
-rw-r--r-- 1 root root 678260987 Dec 11 08:24 ratings.csv
-rw-r--r-- 1 root root  38810332 Dec 11 08:24 tags.csv
